In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
df_api = pd.read_csv('https://data.sfgov.org/resource/yd5s-bd6e.csv')
print('Load data finished')

Load data finished


In [3]:
df_api.head()

,:@computed_region_26cr_cadq,:@computed_region_6qbp_sg9q,:@computed_region_ajp5_b2md,:@computed_region_bh8s_q3mv,:@computed_region_fyvs_ahh9,:@computed_region_p5aj_wyqh,:@computed_region_qgnn_b9vv,:@computed_region_rxqg_mtj9,:@computed_region_yftq_j783,_1_bedroom_units,_2_bedroom_units,_3_bedroom_units,_4_bedroom_units,_5_bedroom_or_larger_units,affordable_beds,affordable_units,disabled_units,family_units,homeless_units,latitude,longitude,losp_units,neighborhood,planning_neighborhood,project_address,project_id,project_location,project_location_address,project_location_city,project_location_state,project_location_zip,project_name,project_sponsor,senior_units,single_room_occupancy_units,street_name,street_number,street_type,studio_units,supervisor_district,tay_units,total_beds,total_units,units_at_120_ami,units_at_20_ami,units_at_30_ami,units_at_50_ami,units_at_60_ami,units_at_80_ami,units_greater_than_120_ami,year_affordability_began,year_building_constructed
0,9.0,88.0,1.0,58.0,1.0,3.0,2.0,8.0,10.0,7.0,36.0,19.0,NaN,NaN,0,62,0,NaN,NaN,37.719645,-122.384831,NaN,Bayview Hunters Point,Bayview,94124,2012-003,POINT (-122.384830980999 37.719645207),NaN,NaN,NaN,NaN,Alice Griffith - Phase 3B (Block 1B),"Double Rock Ventures, LLC",NaN,NaN,Arelious Walker,2500,Drive,NaN,10,NaN,0,62,0.0,0,0.0,62,0.0,0.0,0.0,2018,2018
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,16,0,0,0.0,0.0,NaN,NaN,0.0,Confidential,Confidential,00000,1993-002,NaN,NaN,NaN,NaN,NaN,Asian Women's Shelter,Mei Fong And Associates,0.0,0.0,Confidential,0,Yes,0.0,6,0.0,16,5,0.0,0,0.0,0,16.0,0.0,0.0,1994,1994
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,26,0,0,0.0,0.0,NaN,NaN,0.0,Confidential,Confidential,00000,2010-023,NaN,NaN,NaN,NaN,NaN,Gum Moon Residence Hall,Gum Moon,0.0,0.0,Confidential,0,Yes,0.0,3,0.0,30,33,0.0,0,0.0,26,0.0,0.0,0.0,2012,1911
3,10.0,20.0,36.0,28852.0,36.0,10.0,5.0,9.0,14.0,0.0,0.0,0.0,0.0,0.0,0,89,0,89.0,10.0,37.784500,-122.410263,0.0,Tenderloin,Downtown/Civic Center,94102,1990-003,POINT (-122.4102631 37.7845001),NaN,NaN,NaN,NaN,William Penn Hotel,Chinatown Community Development Center,0.0,89.0,Eddy,160,St,0.0,6,0.0,0,91,0.0,0,0.0,89,0.0,0.0,0.0,1993,1909
4,9.0,86.0,1.0,58.0,1.0,3.0,2.0,8.0,10.0,53.0,0.0,0.0,0.0,0.0,0,53,0,0.0,0.0,37.727093,-122.393120,0.0,Bayview Hunters Point,Bayview,94124,1995-015,POINT (-122.3931198 37.7270927),NaN,NaN,NaN,NaN,Geraldine Johnson Manor,Bridge Housing / San Francisco Housing Develop...,53.0,0.0,3rd,5545,St,0.0,10,0.0,0,54,0.0,0,0.0,53,0.0,0.0,0.0,1998,1998


In [4]:
col_names = df_api.columns
print(col_names)

Index([':@computed_region_26cr_cadq', ':@computed_region_6qbp_sg9q',
       ':@computed_region_ajp5_b2md', ':@computed_region_bh8s_q3mv',
       ':@computed_region_fyvs_ahh9', ':@computed_region_p5aj_wyqh',
       ':@computed_region_qgnn_b9vv', ':@computed_region_rxqg_mtj9',
       ':@computed_region_yftq_j783', '_1_bedroom_units', '_2_bedroom_units',
       '_3_bedroom_units', '_4_bedroom_units', '_5_bedroom_or_larger_units',
       'affordable_beds', 'affordable_units', 'disabled_units', 'family_units',
       'homeless_units', 'latitude', 'longitude', 'losp_units', 'neighborhood',
       'planning_neighborhood', 'project_address', 'project_id',
       'project_location', 'project_location_address', 'project_location_city',
       'project_location_state', 'project_location_zip', 'project_name',
       'project_sponsor', 'senior_units', 'single_room_occupancy_units',
       'street_name', 'street_number', 'street_type', 'studio_units',
       'supervisor_district', 'tay_units', 'tota

In [5]:
df_api.shape

(354, 52)

In [6]:
df_rental=df_api[['project_name','project_address','year_building_constructed',
                   'street_name','street_number','neighborhood','planning_neighborhood',
                   'latitude','longitude']]
df_rental.head(10)

,project_name,project_address,year_building_constructed,street_name,street_number,neighborhood,planning_neighborhood,latitude,longitude
0,Alice Griffith - Phase 3B (Block 1B),94124,2018,Arelious Walker,2500,Bayview Hunters Point,Bayview,37.719645,-122.384831
1,Asian Women's Shelter,00000,1994,Confidential,0,Confidential,Confidential,NaN,NaN
2,Gum Moon Residence Hall,00000,1911,Confidential,0,Confidential,Confidential,NaN,NaN
3,William Penn Hotel,94102,1909,Eddy,160,Tenderloin,Downtown/Civic Center,37.784500,-122.410263
4,Geraldine Johnson Manor,94124,1998,3rd,5545,Bayview Hunters Point,Bayview,37.727093,-122.393120
5,1028 Howard,94103,1994,Howard,1028,South of Market,South of Market,37.779469,-122.407921
6,4042 - 4048 Fulton Street,94118,1921,Fulton,4042,Outer Richmond,Inner Richmond,37.773153,-122.475399
7,344-348 Precita Avenue,94110,1900,Precita,344- 348,Bernal Heights,Bernal Heights,37.746650,-122.412688
8,125 Mason Street,94102,2008,Mason,125,Tenderloin,Downtown/Civic Center,37.784805,-122.409744
9,Walden House,94117,1916,Buena Vista,815,Haight Ashbury,Haight Ashbury,37.769341,-122.443077


In [7]:
df_rental.dtypes

project_name                  object
project_address               object
year_building_constructed      int64
street_name                   object
street_number                 object
neighborhood                  object
planning_neighborhood         object
latitude                     float64
longitude                    float64
dtype: object

In [8]:
df_rental = df_rental[df_rental['year_building_constructed']>=2005]
df_data = df_rental.dropna()
df_data.head(10)

,project_name,project_address,year_building_constructed,street_name,street_number,neighborhood,planning_neighborhood,latitude,longitude
0,Alice Griffith - Phase 3B (Block 1B),94124,2018,Arelious Walker,2500,Bayview Hunters Point,Bayview,37.719645,-122.384831
8,125 Mason Street,94102,2008,Mason,125,Tenderloin,Downtown/Civic Center,37.784805,-122.409744
10,Martin Luther King-Marcus Garvey Square Cooper...,94115,2011,Eddy,1680,Western Addition,Western Addition,37.781597,-122.434860
19,Richardson Apartments (Parcel G),94102,2011,Fulton,365,Hayes Valley,Downtown/Civic Center,37.778492,-122.422958
20,1100 Ocean,94112,2015,Ocean,1100,West of Twin Peaks,West of Twin Peaks,37.725575,-122.454155
28,Friendship House,94103,2005,Julian,56,Mission,Mission,37.767296,-122.421402
31,2175 Market,94114,2013,Market,2175,Castro/Upper Market,Castro/Upper Market,37.766285,-122.429916
33,Hunters View Phase IIB (Block 10),94124,2017,Middle Point,112,Bayview Hunters Point,Bayview,37.735226,-122.380759
36,Octavia Court,94102,2010,Octavia,261,Hayes Valley,Western Addition,37.774601,-122.424461
51,Edith Witt Senior Community,94103,2008,9th,66,South of Market,South of Market,37.776009,-122.415817


In [9]:
df_data.shape

(72, 9)

In [10]:
# create map of SanFran using latitude and longitude values
map_sf = folium.Map(location=[37.77, -122.42], zoom_start=15)

# add markers to map
for lat, lng, label in zip(df_data['latitude'], df_data['longitude'], df_data['project_name']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sf)

# add pop-up text to each marker on the map
latitudes = list(df_data['latitude'])
longitudes = list(df_data['longitude'])
labels = list(df_data['project_name'])

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(map_sf) 
    
map_sf